In [1]:
import pandas as pd
from bulletin.commom.utils import auto_fit_columns
import codecs

pd.set_option('display.max_columns', None)

information_schema = pd.read_csv('information_schema.csv', usecols=['table_name','column_name','udt_name'])
constraint = pd.read_csv('constraint.csv')

# information_schema = pd.merge(information_schema,constraint[['table_name','column_name','foreign_table_name','foreign_column_name']],on=['table_name','column_name'], how='left')
information_schema = information_schema.drop(index=information_schema.loc[information_schema['table_name']=='notificacao'].index)

In [2]:
notificacao = pd.read_csv('notificacao.csv', sep=';')

notificacao = notificacao.loc[notificacao['importante']=='Sim'].drop(columns=['importante'])
notificacao = pd.merge(notificacao,constraint[['table_name','column_name','foreign_table_name','foreign_column_name']],on=['table_name','column_name'], how='left').fillna('')
notificacao['ref'] = ""
notificacao.loc[(notificacao['table_name'] == notificacao['foreign_table_name']) & (notificacao['column_name']==notificacao['foreign_column_name']),'ref'] = f"[pk]"
notificacao

,table_name,column_name,udt_name,foreign_table_name,foreign_column_name,ref
0,notificacao,criterio_classificacao,int4,criterio_classificacao,id,
1,notificacao,evolucao,int4,evolucao,id,
2,notificacao,raca_cor,int4,raca_cor,id,
3,notificacao,perda_olfato_paladar,int4,sim_nao,id,
4,notificacao,cod_cbo,int4,cbo,codigo,
...,...,...,...,...,...,...
96,notificacao,ibge_residencia,varchar,municipio,co_municipio,
97,notificacao,cep_residencia,varchar,,,
98,notificacao,logradouro_residencia,varchar,,,
99,notificacao,numero_residencia,varchar,,,


In [3]:
with codecs.open('tables.dbml',"w","utf-8-sig") as out:

    for table_name, table in information_schema.groupby('table_name'):
        if any(table_name in n for n in notificacao['foreign_table_name'].values):
            out.write(f"Table {table_name}" + " {\n")
            for column_name, udt_name in zip(table['column_name'],table['udt_name']):
                if column_name in constraint.loc[constraint['table_name'] == table_name, 'foreign_column_name'].values:
                    out.write(f"\t {column_name} {udt_name} [pk]\n")
                    notificacao.loc[notificacao['foreign_table_name']==table_name,'ref'] = f"[ref: - {table_name}.{column_name}]"
                else:
                    out.write(f"\t {column_name} {udt_name}\n")
            out.write("}\n\n")


    out.write(f"Table notificacao" + " {\n")
    for column_name, udt_name,ref in zip(notificacao['column_name'],notificacao['udt_name'],notificacao['ref']):
        out.write(f"\t {column_name} {udt_name} {ref if ref != None else ''}\n")
    out.write("}\n\n")